# Lib install

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
%%bash
cd '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger'
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz

wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/french.par.gz

sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

In [ ]:
!pip install --upgrade treetaggerwrapper
!pip install googletrans==4.0.0-rc1

# Lib import

In [6]:
#load libs
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pprint   # For proper print of sequences.
import treetaggerwrapper
from googletrans import Translator
from nltk.tokenize import sent_tokenize 
import glob

In [7]:
text_files_path = '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/OEBPS'

In [8]:
tagged_files_path = '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/tagged_french_version'

In [120]:
complete_bible_path = '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/complete_french_bible'

# Functions (old)

In [92]:
# lots of commented debug code here
# as well as examples of each fucntion

# get all dataset filenames at path
def get_filename_list_at(input_path):
  filename_list = []
  for file in glob.glob(input_path+"/*", recursive = True):
    filename_list.append(file)
  return filename_list

# filename_list = get_filename_list_at(text_files_path)
# remove line number from parsed text
def clean_text(text):
  return re.sub(r'^[0-9]+', '', text)


def simple_replace(text, target, replace):
  return re.sub(r''+target, replace, text)



def get_text_lines_from_page_by_filename(filename):
  text_lines = []
  with open(filename) as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  fp.close()
  
  # each paragraph is only one line long
  for paragraph in soup.find_all('p'):
    text_lines.append((clean_text(paragraph.get_text())))
    
  return text_lines

# for filename in tqdm(filename_list):
#   text_lines = get_text_lines_from_page_by_filename(filename)
#   print(text_lines)
#   break #remove this to handle all files
# test_sentence = text_lines[0]
# test_sentence = test_sentence.split('.')[0]
# load english and french tagg trees
tagger_en = treetaggerwrapper.TreeTagger(TAGLANG='en', TAGDIR='/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger') # treetagger is the installation dir
tagger_fr = treetaggerwrapper.TreeTagger(TAGLANG='fr', TAGDIR='/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger') # treetagger is the installation dir


#tag french and english strings
def tag_french(french_string):
  tags = tagger_fr.tag_text(french_string)
  return treetaggerwrapper.make_tags(tags)

def tag_english(english_string):
  tags = tagger_en.tag_text(english_string)
  return treetaggerwrapper.make_tags(tags)

def translate_string(input_string):
  translator = Translator()
  translation = translator.translate(input_string)
  return translation.text

# french_string = test_sentence
# english_string = translate_string(french_string)

# french_tags = tag_french(french_string)
# english_tags = tag_english(english_string)
#return True if pro:per "il" is transformable into "iel"
#return False if it isn't
def is_pro_per_transformable(english_string):
  for tag in tag_english(english_string):
    # check for out of range tags class "NotTag"
    # print(type(tag))
    if ((isinstance(tag, treetaggerwrapper.Tag)) and (tag[1] == 'PP')):
      # print(tag[0].lower())
      if (tag[0].lower() in ['he', 'they']):
        return True
  return False
                  
# english_string = translate_string(test_sentence)
# is_pro_per_transformable(english_string)
#replace nth word in sentence
def replace_word_at(input_string, replace_string, input_index):
  word_list = input_string.split(' ')
  word_list[input_index] = replace_string
  return ' '.join(word_list).strip()


# input_string = 'il est allé à la plage.'
# replace_string = 'montagne'
# input_index = 5
# print(input_string)
# print(replace_word_at(input_string, replace_string, input_index))
def split_line_into_sentences(text_line):
  return sent_tokenize(text_line, language='french') 

def get_new_pro_per(tag_label, input_word, french_string):

  word = 'none'

  if (tag_label == 'PRO:PER'):
    # print('inside ------------------------')
    # print(word)
    # print('inside ------------------------')
    if ((input_word.lower() == 'elle') or (input_word.lower() == 'elles')):
      if input_word == 'elle':
        # print('replace iel')
        return 'iel'
      elif input_word == 'Elle':
        # print('replace Iel')
        return 'Iel'
      elif input_word == 'elles':
        # print('replace iels')
        return 'iels'
      elif input_word == 'Elles':
        # print('replace Iels')
        return 'Iels'
    elif((input_word.lower() == 'il') or (input_word.lower() == 'ils')):
      # check the real nature of "il"
      english_string = translate_string(french_string)
      # print(english_string)
      is_transformable = is_pro_per_transformable(english_string)
      # print(english_string + ' ' + str(is_transformable))
      if (is_transformable):
        if input_word == 'il':
          # print('replace iel')
          return 'iel'
        elif input_word == 'Il':
          # print('replace Iel')
          return 'Iel'
        elif input_word == 'ils':
          # print('replace iels')
          return 'iels'
        elif input_word == 'Ils':
          # print('replace Iels')
          return 'Iels'
          
  return word

# french_string = 'Il pleut, ce qui n\'est pas bon signe pour elle.'
# french_tags = tag_french(french_string)
# for tag in french_tags:
#   if(get_new_pro_per(tag[1], tag[0], french_string) != 'none'):
#     print('ha')

# process pos tags for pro_per replacement
# returns transformed sentence or original
def apply_pos_transformer(french_tags, french_string):
  new_pro_per = 'none'

  for position,tag in enumerate(french_tags):
    word = tag[0]
    tag_label = tag[1]
    # print(word)
    # print(tag_label)
    
    new_pro_per = get_new_pro_per(tag_label, word, french_string)
    if (new_pro_per != 'none'):
      # print(position)
      return simple_replace(french_string, word, new_pro_per)


  # print(french_string, new_pro_per, position)
  # print(new_pro_per)
    
  # return original sentence if no changes are needed
  
  return french_string


# french_string = 'Il pleut, ce qui n\'est pas bon signe pour elle.'
# french_tags = tag_french(french_string)
# apply_pos_transformer(french_tags, french_string)
import pickle


def save_page_to_pickle(content_to_save, page_counter):
  with open('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/first_export/page_'+str(page_counter)+'.txt', 'wb') as fp:
    pickle.dump(content_to_save, fp)
  fp.close()

def load_page_from_pickle(page_counter):
  with open('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/first_export/page_'+str(page_counter)+'.txt', 'rb') as fp:
    return pickle.load(fp)


# list_1 = ['this is a test', 'this is a test 2']
# save_page_to_pickle(list_1, 3)
# load_page_from_pickle(3)


def separate_line_number(text):
  # return re.sub(r'^[0-9]+', '', text)
  return re.sub(r'^([0-9]+)', r'\1 - ', text)

# New functions

In [ ]:
# tagger_fr = treetaggerwrapper.TreeTagger(TAGLANG='fr', TAGDIR='/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger') # treetagger is the installation dir
# tags = tagger_fr.tag_text(french_string)



#get list of filenames corresponding to all bible pages
filename_list = get_filename_list_at(text_files_path)

#get all text lines from all pages
page_list = []
for filename in tqdm(filename_list):
  text_lines = get_text_lines_from_page_by_filename(filename)
  # print(text_lines)
  page_list.append(text_lines)
  break #remove this to handle all files

# # a page contains text lines
# output_pages = []
# page_counter = 0
# for page in tqdm(page_list):
#   page_counter = page_counter + 1
#   output_text_line = []
#   # a text line contains sentences
#   for line_number, text_line in enumerate(page):
#     output_sentence = []
#     # a sentence contains words
#     for sentence in split_line_into_sentences(text_line):
#       # print(str(line_number) + ' - ' + sentence)
#       french_tags = tag_french(sentence)
#       new_sentence = apply_pos_transformer(french_tags, sentence)
#       output_sentence.append(new_sentence)
#       # print(str(line_number) + ' - ' + new_sentence)
#       # break
#     output_text_line.append(' '.join(output_sentence))
#     # break
#   # output_pages.append(output_text_line)
#   save_page_to_pickle(output_text_line, page_counter)
#   # break

page_list

In [77]:
def order_bible_pages(filename_list):
  filename_df = pd.DataFrame(filename_list, columns=['raw_filename_and_path'])
  # filename_df['filename'] = 


  filename_df['filename_plus_extension'] = filename_df.raw_filename_and_path.str.split('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/OEBPS/', expand=True)[1]
  filename_df['filename'] = filename_df.filename_plus_extension.str.split('.xhtml', expand=True)[0]

  # drop non-xhtml files
  filename_df = filename_df[filename_df['filename_plus_extension'].str.contains('.xhtml')]
  filename_df = filename_df[~filename_df['filename_plus_extension'].str.contains('title_page')]
  del filename_df['filename_plus_extension']


  #replace Str page number by float composed page number
  filename_df[['chapter_number', 'page_number']] = filename_df.filename.str.split('_', expand=True)
  filename_df['chapter_number'] = filename_df['chapter_number'].astype(float)
  filename_df['page_number'] = filename_df['page_number'].astype(float)


  filename_df = filename_df.sort_values(['chapter_number', 'page_number'])
  filename_df.reset_index(drop=True, inplace=True)
  return filename_df


filename_df = order_bible_pages(filename_list)
filename_df

,raw_filename_and_path,filename,chapter_number,page_number
0,/content/drive/MyDrive/Colab Notebooks/ressour...,01_1,1.0,1.0
1,/content/drive/MyDrive/Colab Notebooks/ressour...,01_2,1.0,2.0
2,/content/drive/MyDrive/Colab Notebooks/ressour...,01_3,1.0,3.0
3,/content/drive/MyDrive/Colab Notebooks/ressour...,01_4,1.0,4.0
4,/content/drive/MyDrive/Colab Notebooks/ressour...,01_5,1.0,5.0
...,...,...,...,...
1184,/content/drive/MyDrive/Colab Notebooks/ressour...,66_18,66.0,18.0
1185,/content/drive/MyDrive/Colab Notebooks/ressour...,66_19,66.0,19.0
1186,/content/drive/MyDrive/Colab Notebooks/ressour...,66_20,66.0,20.0
1187,/content/drive/MyDrive/Colab Notebooks/ressour...,66_21,66.0,21.0


In [106]:
def get_text_lines_from_page_by_filename2(filename):
  text_lines = []
  with open(filename) as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  fp.close()
  
  # each paragraph is only one line long
  for paragraph in soup.find_all('p'):
    text_lines.append(separate_line_number(paragraph.get_text()))
    
  return text_lines

# Convert HTML Bible to a single long text

In [146]:
#get list of filenames corresponding to all bible pages
filename_list = get_filename_list_at(text_files_path)
filename_df = order_bible_pages(filename_list)

#get all text lines from all pages
page_df_list = []
page_number = 1
for filename in tqdm(filename_df.raw_filename_and_path):
  text_lines = get_text_lines_from_page_by_filename2(filename)
  # print(text_lines)
  text_lines = pd.DataFrame(text_lines, columns=['line_and_number'])
  text_lines['page_number'] = page_number
  page_number = page_number + 1
  page_df_list.append(text_lines)
  # break #remove this to handle all files


complete_bible = pd.concat(page_df_list)

  0%|          | 0/1189 [00:00<?, ?it/s]

In [147]:
complete_bible_path = '/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/complete_french_bible/'

In [159]:
filename_df['global_page_number'] = filename_df.page_number.index+1
filename_df

,raw_filename_and_path,filename,chapter_number,page_number,global_page_number
0,/content/drive/MyDrive/Colab Notebooks/ressour...,01_1,1.0,1.0,1
1,/content/drive/MyDrive/Colab Notebooks/ressour...,01_2,1.0,2.0,2
2,/content/drive/MyDrive/Colab Notebooks/ressour...,01_3,1.0,3.0,3
3,/content/drive/MyDrive/Colab Notebooks/ressour...,01_4,1.0,4.0,4
4,/content/drive/MyDrive/Colab Notebooks/ressour...,01_5,1.0,5.0,5
...,...,...,...,...,...
1184,/content/drive/MyDrive/Colab Notebooks/ressour...,66_18,66.0,18.0,1185
1185,/content/drive/MyDrive/Colab Notebooks/ressour...,66_19,66.0,19.0,1186
1186,/content/drive/MyDrive/Colab Notebooks/ressour...,66_20,66.0,20.0,1187
1187,/content/drive/MyDrive/Colab Notebooks/ressour...,66_21,66.0,21.0,1188


In [174]:
complete_bible

complete_bible_with_metadata = pd.merge(left=complete_bible, right=filename_df, left_on='page_number', right_on='global_page_number')
del complete_bible_with_metadata['page_number_x']
del complete_bible_with_metadata['raw_filename_and_path']

complete_bible_with_metadata[['line_number', 'sentence']] = complete_bible_with_metadata.line_and_number.str.split(' - ', 1,expand=True)
del complete_bible_with_metadata['line_and_number']
del complete_bible_with_metadata['filename']

# complete_bible_with_metadata.columns[2] = 'page_number'
complete_bible_with_metadata.rename(index=str,columns={'page_number_y':'page_number'},inplace=True)

complete_bible_with_metadata = complete_bible_with_metadata[['chapter_number', 'page_number', 'line_number', 'sentence']]
complete_bible_with_metadata

,chapter_number,page_number,line_number,sentence
0,1.0,1.0,1,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,Dieu dit: Que la lumière soit! Et la lumière fut.
3,1.0,1.0,4,Dieu vit que la lumière était bonne; et Dieu s...
4,1.0,1.0,5,"Dieu appela la lumière jour, et il appela les ..."
...,...,...,...,...
31097,66.0,22.0,17,Et l'Esprit et l'épouse disent: Viens. Et que ...
31098,66.0,22.0,18,Je le déclare à quiconque entend les paroles d...
31099,66.0,22.0,19,et si quelqu'un retranche quelque chose des pa...
31100,66.0,22.0,20,"Celui qui atteste ces choses dit: Oui, je vien..."


In [175]:
complete_bible_path+'complete_bible.csv'

'/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/complete_french_bible/complete_bible.csv'

In [176]:
complete_bible_with_metadata.to_csv(complete_bible_path+'complete_bible.csv')

# How to ASYNC TreeTag a big text file. 

In [ ]:
import sys
import time

JOBSCOUNT = 100

def tree_tag_sentences(sentences):
    start = time.time()
    import treetaggerpoll

    # tagger_fr = treetaggerwrapper.TreeTagger(TAGLANG='fr', TAGDIR='/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger') # treetagger is the installation dir
    # Note: print() have been commented, you may uncomment them to see progress.
    p = treetaggerpoll.TaggerProcessPoll(workerscount=10, TAGLANG="fr", TAGDIR='/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/treetagger')
    res = []

    # text = "This is Mr John's own house, it's very nice. " * 40
    text = sentences

    print("Creating jobs")
    for i in range(JOBSCOUNT):
        # print("   Job", i)
        res.append(p.tag_text_async(text=text, numlines=len(text)))

    print("Waiting for jobs to complete")
    for i, r in tqdm(enumerate(res)):
        # print("   Job", i)
        r.wait_finished()
        # print(str(r.result)[:50])
        # res[i] = None   # Loose Job reference - free it.
        res[i] = r.result

    p.stop_poll()
    print("Finished after {:0.2f} seconds elapsed".format(time.time() - start))

    return res

result = tree_tag_sentences(page_list)
result

"This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is Mr John's own house, it's very nice. This is M

# Main